In [4]:
import json

In [5]:
business = []
with open("../dataset/business_all.json", "r") as file:
    for line in file:
        business.append(json.loads(line))

checkin = []
with open("../dataset/checkin_all.json", "r") as file:
    for line in file:
        checkin.append(json.loads(line))

train_review = []
with open("../dataset/review_train.json", "r") as file:
    for line in file:
        train_review.append(json.loads(line))

test_review = []
with open("../dataset/review_test.json", "r") as file:
    for line in file:
        test_review.append(json.loads(line))

user = []
with open("../dataset/user_all.json", "r") as file:
    for line in file:
        user.append(json.loads(line))
        

In [6]:
### baseline
train_average_stars = sum(d["stars"] for d in train_review) / len(train_review)
mse = sum((d["stars"] - train_average_stars) ** 2 for d in test_review) / len(test_review)
print(mse)

1.490833578439748
